# Clustering with K-means Emotion

### 1.Est-que les clusters obtenue avec kmeans reflete les groupements de texte par émotions ?

non, car il va chercher d'autre parametre pour les labéliser



### 3.quelle est le nombre de cluster optimal au regard de la métrique "silhouette" ? de l'inertie ?  est ce que l'inertie est corrélé à la mesure de silhouette ?

on va perdre en justesse de labélisation mais on aura de meilleur score. 


In [68]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_digits
from sklearn.cluster import KMeans

import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

import nltk
#nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer

In [69]:
df=pd.read_csv('/home/cecilia/Documents/brief/brief_emotions/Emotion_final.csv')

In [70]:
df.head(10)

,Text,Emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger
5,ive been feeling a little burdened lately wasn...,sadness
6,ive been taking or milligrams or times recomme...,surprise
7,i feel as confused about life as a teenager or...,fear
8,i have been with petronas for years i feel tha...,happy
9,i feel romantic too,love


In [71]:
stopword = nltk.corpus.stopwords.words('english')
targets = df['Emotion']
corpus = df['Text']
vec = CountVectorizer(stop_words=stopword)
X =vec.fit_transform(corpus)

In [76]:
kmeans = KMeans(n_clusters=6, random_state=300)
kmeans_fitted = kmeans.fit_transform(X)
kmeans_fitted

array([[1.8162864 , 1.75741784, 2.29923407, 1.87625853, 2.0038115 ,
        1.421019  ],
       [3.36214975, 2.9998929 , 3.0636594 , 3.16381222, 2.81615709,
        3.16628275],
       [2.65476773, 2.65200761, 2.62099514, 2.71829733, 2.83200146,
        2.43207906],
       ...,
       [2.34707557, 1.74938894, 2.33812001, 2.1608522 , 2.03358957,
        2.03616594],
       [3.07346249, 2.63247155, 3.06709736, 2.92199873, 2.83861699,
        2.8461174 ],
       [2.12407844, 1.44646865, 2.11569678, 1.9173874 , 1.77302626,
        1.77528019]])

In [77]:
#encodage des données
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(targets)

### 2.comparer les résultats de plusieurs modéles dans un tableaux avec le temps, l'inertie, la NMI, la AMI, l'ARI Index et la silhouette.

In [78]:
from time import time
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler


def bench_k_means(kmeans, name, data, labels):
    """Benchmark to evaluate the KMeans initialization methods.

    Parameters
    ----------
    kmeans : KMeans instance
        A :class:`~sklearn.cluster.KMeans` instance with the initialization
        already set.
    name : str
        Name given to the strategy. It will be used to show the results in a
        table.
    data : ndarray of shape (n_samples, n_features)
        The data to cluster.
    labels : ndarray of shape (n_samples,)
        The labels used to compute the clustering metrics which requires some
        supervision.
    """
    t0 = time()
    estimator = make_pipeline(StandardScaler(with_mean=False), kmeans).fit(data)
    fit_time = time() - t0
    results = [name, fit_time, estimator[-1].inertia_]

    # Define the metrics which require only the true labels and estimator
    # labels
    clustering_metrics = [
        metrics.homogeneity_score,
        metrics.completeness_score,
        metrics.v_measure_score,
        metrics.adjusted_rand_score,
        metrics.adjusted_mutual_info_score,
    ]
    results += [m(labels, estimator[-1].labels_) for m in clustering_metrics]

    # The silhouette score requires the full dataset
    results += [
        metrics.silhouette_score(data, estimator[-1].labels_,
                                 metric="euclidean",)
    ]

    # Show the results
    formatter_result = ("{:9s}\t{:.3f}s\t{:.0f}\t{:.3f}\t{:.3f}"
                        "\t{:.3f}\t{:.3f}\t{:.3f}\t{:.3f}")
    print(formatter_result.format(*results))

In [79]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

print(82 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')

kmeans = KMeans(init="k-means++", n_clusters=6, n_init=4,
                random_state=0)
bench_k_means(kmeans=kmeans, name="k-means++", data=X, labels=y)

kmeans = KMeans(init="random", n_clusters=6, n_init=4, random_state=0)
bench_k_means(kmeans=kmeans, name="random", data=X, labels=y)

clf = TruncatedSVD(100)
X = clf.fit_transform(X)

pca = PCA(n_components=6).fit(X)
kmeans = KMeans(init=pca.components_, n_clusters=6, n_init=1)
bench_k_means(kmeans=kmeans, name="PCA-based", data=X, labels=y)

print(82 * '_')

__________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	silhouette
k-means++	0.343s	408284055	0.000	0.119	0.000	-0.000	-0.000	0.200
random   	0.286s	408938893	0.000	0.073	0.001	-0.000	0.000	-0.090
PCA-based	0.115s	2059144	0.006	0.006	0.006	0.010	0.006	0.122
__________________________________________________________________________________


In [83]:
#tfidf

print(82 * '_')
print('init\t\ttime\tinertia\thomo\tcompl\tv-meas\tARI\tAMI\tsilhouette')

kmeans = KMeans(init="k-means++", n_clusters=6, n_init=4, random_state=0)
bench_k_means(kmeans=kmeans, name="k-means++", data=X, labels=y)

kmeans = KMeans(init="random", n_clusters=6, n_init=4, random_state=0)
bench_k_means(kmeans=kmeans, name="random", data=X, labels=y)

from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf_vectorizer = TfidfVectorizer()
tf_idf = tf_idf_vectorizer.fit_transform(corpus)

clf = TruncatedSVD()
Xpca = clf.fit_transform(tf_idf)
XX = PCA(2).fit_transform(Xpca)
kmeans = KMeans(n_clusters=6, n_init=1)
bench_k_means(kmeans=kmeans, name="PCA-based", data=XX, labels=y)
from sklearn.feature_extraction.text import TfidfVectorizer
print(82 * '_')

__________________________________________________________________________________
init		time	inertia	homo	compl	v-meas	ARI	AMI	silhouette
k-means++	0.447s	2051501	0.001	0.002	0.001	-0.001	0.000	0.103
random   	0.397s	2039742	0.005	0.008	0.006	0.008	0.006	0.153
PCA-based	0.034s	8327	0.003	0.003	0.003	0.003	0.003	0.360
__________________________________________________________________________________
